<a href="https://colab.research.google.com/github/JoseAg02/Actividades/blob/main/PIA_IAyRN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Jose de Jesus Aguilar 1955193

*Mauricio Alejandro Reyna Miranda 2077482

*Ivan Martínez Fabian 1920753








In [ ]:
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import numpy as np
import glob
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d hariharanalm/river-vs-lake

river-vs-lake.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile

# Ruta al archivo ZIP descargado
zip_file_path = '/content/river-vs-lake.zip'

# Ruta a la carpeta donde deseas descomprimir el archivo
extracted_folder_path = '/content/Imagenes'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [ ]:
carpeta_principal = '/content/Imagenes/River vs Lake'
patron = os.path.join(carpeta_principal, '*')
subcarpetas = glob.glob(patron)

imagenes = []  # Lista para almacenar las imágenes redimensionadas
etiquetas = []  # Lista para almacenar las etiquetas correspondientes

z = 0
for subcarpeta in subcarpetas:
    if os.path.isdir(subcarpeta):
        print('Subcarpeta:', subcarpeta)

        for x in glob.iglob(f'{subcarpeta}/*jpg'):
            print(x)
            test_image = cv2.imread(x)

            img_resized = cv2.resize(test_image, (32, 32))

            arreglo_imagen = np.array(img_resized)
            imagenes.append(arreglo_imagen)
            etiquetas.append(z)

        # Después de procesar una subcarpeta, incrementa la etiqueta para la siguiente subcarpeta
        z += 1



Subcarpeta: /content/Imagenes/River vs Lake/river water
/content/Imagenes/River vs Lake/river water/Image_64.jpg
/content/Imagenes/River vs Lake/river water/Image_92.jpg
/content/Imagenes/River vs Lake/river water/Image_7.jpg
/content/Imagenes/River vs Lake/river water/Image_82.jpg
/content/Imagenes/River vs Lake/river water/Image_60.jpg
/content/Imagenes/River vs Lake/river water/Image_43.jpg
/content/Imagenes/River vs Lake/river water/Image_48.jpg
/content/Imagenes/River vs Lake/river water/Image_68.jpg
/content/Imagenes/River vs Lake/river water/Image_33.jpg
/content/Imagenes/River vs Lake/river water/Image_96.jpg
/content/Imagenes/River vs Lake/river water/Image_44.jpg
/content/Imagenes/River vs Lake/river water/Image_24.jpg
/content/Imagenes/River vs Lake/river water/Image_42.jpg
/content/Imagenes/River vs Lake/river water/Image_46.jpg
/content/Imagenes/River vs Lake/river water/Image_88.jpg
/content/Imagenes/River vs Lake/river water/Image_16.jpg
/content/Imagenes/River vs Lake/r

In [ ]:
etiquetas = np.array(etiquetas)

In [ ]:
len(imagenes)

177

In [ ]:
len(etiquetas)

177

In [ ]:
images_train, images_test, labels_train, labels_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
Total params: 56320 (220.00 KB)
Trainable params: 5632

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten_2 (Flatten)         (None, 1024)             

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(images_train, labels_train, epochs=10,
                    validation_data=(images_test, labels_test))

ValueError: ignored

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)